# ProductHunt Products from the January 2023

In [43]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from matplotlib.dates import DateFormatter
import networkx as nx
from adjustText import adjust_text
import plotly.graph_objects as go
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls


## Setting up a Plotly Server

In [44]:
username='Mehdi_Ali_Mehri'
api_key='UiJrOiViVQ4PYcJVlZuO'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)

# Import & Analyse the raw dataset

In [2]:
# Step 1: Load the Dataset
# df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Stats_Year_2022.csv")
Jan_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Jan_2023_Fix_Cleaned.csv")
Feb_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Feb_2023_Cleaned.csv")
Mar_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Mar_2023_Cleaned.csv")
# df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Feb_2023_Cleaned.csv")

# Assuming you have two DataFrames named 'df1' and 'df2'
df = pd.concat([Jan_df, Feb_df, Mar_df], ignore_index=True)

df.head()

,id,name,votesCount,reviewsRating,reviewsCount,commentsCount,createdAt,totalCount,topics
0,377488,Perplexity.ai,214,0.0,0,19,2023-02-01T01:10:23Z,20,Search
1,377270,Image Editor AI,46,0.0,0,11,2023-02-01T08:00:00Z,15,"Design Tools, Marketing, Tech"
2,377466,RoboHelper,21,0.0,0,5,2023-01-31T21:41:01Z,3,Productivity
3,376463,Dover Autopilot,1118,0.0,0,134,2023-01-31T08:03:56Z,20,"Hiring, Artificial Intelligence"
4,377244,Unlimited Voice Transcription with API,873,0.0,0,229,2023-01-31T08:12:31Z,20,"Productivity, Privacy"


In [3]:
df = df.drop('id', axis=1)
df.shape


(745423, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745423 entries, 0 to 745422
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           745423 non-null  object 
 1   votesCount     745423 non-null  int64  
 2   reviewsRating  745423 non-null  float64
 3   reviewsCount   745423 non-null  int64  
 4   commentsCount  745423 non-null  int64  
 5   createdAt      745423 non-null  object 
 6   totalCount     745423 non-null  int64  
 7   topics         745423 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 45.5+ MB


# Step 2: Data Cleaning & Manipulation

In [5]:
# Check for missing values
print(df.isnull().sum())

name             0
votesCount       0
reviewsRating    0
reviewsCount     0
commentsCount    0
createdAt        0
totalCount       0
topics           0
dtype: int64


In [6]:
df.describe()

,votesCount,reviewsRating,reviewsCount,commentsCount,totalCount
count,745423.000000,745423.0,745423.0,745423.000000,745423.000000
mean,98.227410,0.0,0.0,22.310007,6.894432
std,210.094115,0.0,0.0,61.054732,7.283637
min,1.000000,0.0,0.0,0.000000,0.000000
25%,5.000000,0.0,0.0,1.000000,0.000000
50%,30.000000,0.0,0.0,4.000000,4.000000
75%,87.000000,0.0,0.0,15.000000,12.000000
max,5022.000000,0.0,0.0,1060.000000,20.000000


### Remove the reviewRating and reviewsCount columns since they are always 0

In [7]:
df = df.drop('reviewsRating', axis=1)
df = df.drop('reviewsCount', axis=1)
df = df.drop('totalCount', axis=1)

### Remove Duplicates and keep the first appearance

In [8]:
df.drop_duplicates(subset=['name', 'createdAt'], keep='first', inplace=True)

### Change the date column type to DataTime

In [9]:
df['createdAt'] = pd.to_datetime(df['createdAt'])
df.head()

,name,votesCount,commentsCount,createdAt,topics
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,Search
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,"Design Tools, Marketing, Tech"
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,Productivity
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,"Hiring, Artificial Intelligence"
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,"Productivity, Privacy"


### Add a month Column

In [10]:

# Create the "Month" column with the corresponding month
df['Month'] = df['createdAt'].dt.month

# Rearrange the columns to position "Month" after "createdAt"
cols = list(df.columns)
cols.insert(cols.index('createdAt') + 1, cols.pop(cols.index('Month')))
df = df[cols]
df['Month'] = df['createdAt'].dt.strftime('%B')
df.head()


,name,votesCount,commentsCount,createdAt,Month,topics
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,February,Search
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,February,"Design Tools, Marketing, Tech"
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,January,Productivity
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,January,"Hiring, Artificial Intelligence"
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,January,"Productivity, Privacy"


### Adding an Hours column

In [11]:
# Convert the 'createdAt' column to pandas datetime
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Create a new column 'hours' containing only the hours from the 'createdAt' column
df['hours'] = df['createdAt'].dt.hour
df['day'] = df['createdAt'].dt.day

# # Sort the DataFrame by the 'hours' column in ascending order
# New_filtered_df = df.sort_values(by='hours', ascending=True)

# New_filtered_df.head()

## Keeping the rows with months inside of Q1

In [12]:
# df = df[df['Month'] == 'February']

# List of months to keep
months_to_keep = ['January', 'February', 'March']

# Filter the DataFrame to keep only rows with the specified months
df = df[df['Month'].isin(months_to_keep)]

df.head()


,name,votesCount,commentsCount,createdAt,Month,topics,hours,day
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,February,Search,1,1
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,February,"Design Tools, Marketing, Tech",8,1
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,January,Productivity,21,31
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,January,"Hiring, Artificial Intelligence",8,31
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,January,"Productivity, Privacy",8,31


In [13]:
df.shape
output_file = "Q1_Data.csv"
df.to_csv(output_file, index=False)


In [50]:

# Calculate the average votesCount
average_votes_count = df['votesCount'].mean()

# Count products with votesCount greater than the average
products_above_average = df[df['votesCount'] > 500]
count_above_average = len(products_above_average)

# Display the results
print("Average votesCount:", average_votes_count)
print("Number of products with votesCount above average:", count_above_average)

Average votesCount: 75.40401218892839
Number of products with votesCount above average: 263


In [17]:
# Assuming your dataset is stored in a DataFrame called 'df'
average_votes_by_hour = df.groupby('hours')['votesCount'].mean().reset_index()
sorted_averages = average_votes_by_hour.sort_values(by='votesCount', ascending=False)
top_hours = sorted_averages.head(3)
print(top_hours)


   hours  votesCount
8      8  141.753124
7      7  109.696277
9      9   61.687898


In [18]:
# Add a new column to store the count of products for each top hour
top_hours['productCount'] = top_hours['hours'].apply(lambda hour: len(df[df['hours'] == hour]))

print(top_hours)

   hours  votesCount  productCount
8      8  141.753124          2641
7      7  109.696277          1182
9      9   61.687898           314


C:\Users\alime\AppData\Local\Temp\ipykernel_23736\2113425084.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
# Calculate the total count of products
total_products = len(df)

# Group the data by hours and calculate the sum of products for each hour
hourly_product_counts = df.groupby('hours')['name'].count().reset_index()

# Find the top 3 hours
top_hours = hourly_product_counts.sort_values(by='name', ascending=False).head(3)

# Calculate the count of products for the remaining hours
remaining_hours = total_products - top_hours['name'].sum()

# Create a new DataFrame for pie plot
pie_data = pd.concat([top_hours, pd.DataFrame({'hours': ['Other'], 'name': [remaining_hours]})])

# Calculate the percentage for each category
pie_data['percentage'] = (pie_data['name'] / total_products) * 100

# Create hover text with percentage and count information
pie_data['hover_text'] = pie_data.apply(lambda row: f"{row['percentage']:.2f}% ({row['name']} products)", axis=1)

# Create a pie plot using Plotly with custom styling
Top_Hours_Products_fig = px.pie(pie_data, values='percentage', names='hours', title='Percentage of Products by Hours', hover_data=['hover_text'],
             labels={'hours': 'Top Hours'}, hole=0.5)

# Update layout for modern and beautiful styling
Top_Hours_Products_fig.update_traces(textposition='outside', marker=dict(line=dict(color='#ffffff', width=2)))

# Update layout for legend placement and appearance
Top_Hours_Products_fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                  margin=dict(l=0, r=0, b=0, t=30))

# Show the plot
Top_Hours_Products_fig.show()

py.plot(Top_Hours_Products_fig, filename="Top_Hours_Products_fig", auto_open = True)

'https://plotly.com/~Mehdi_Ali_Mehri/1/'

In [14]:
import pandas as pd

# Assuming your existing DataFrame is named 'df' and you want to extract the 'column_name' column
column_name = 'topics'

# Create a new DataFrame with only the desired column
new_df = pd.DataFrame(df[column_name])

# Print the new DataFrame
print(new_df.shape)

# Step 1: Split the topic groups into individual topics
all_topics = new_df['topics'].str.split(', ')

# Step 2: Create a set to store unique topics
unique_topics_set = set()

# Step 3: Add each individual topic to the set to remove duplicates
for topics_list in all_topics:
    unique_topics_set.update(topics_list)

# Step 4: Create a new DataFrame with the unique topics
unique_topics_df = pd.DataFrame({'unique_topics': list(unique_topics_set)})

# Print the new DataFrame with unique topics
print(unique_topics_df.shape)
# Assuming your DataFrame is named 'df' and you want to save it to a CSV file named 'output.csv'
# output_file = 'Topics_Groups.csv'

# # Save the DataFrame to a CSV file
# new_df.to_csv(output_file, index=False)



(7876, 1)
(371, 1)


# Insights Extraction

## Calculate the correlation betwee the voteCounts and commentCounts

In [349]:
correlation_coefficient = df['votesCount'].corr(df['commentsCount'])

# correlation_coefficient = df[['votesCount', 'commentsCount', 'createdAt']].corr()

print("Correlation Coefficient between 'votesCount' and 'commentsCount':", correlation_coefficient)


Correlation Coefficient between 'votesCount' and 'commentsCount': 0.8055273754767142


### Visualize the correlation

In [350]:
# Scatter plot using Plotly
fig = px.scatter(df, x='votesCount', y='commentsCount', title='Correlation between Votes Count and Comments Count')
fig.update_layout(
    xaxis_title='Votes Count',
    yaxis_title='Comments Count',
    template='plotly_white',  # Choose a modern template
    hovermode='closest',
)
fig.show()


## Usage of the Top 3 Topics over time

In [351]:
# Get the frequency of each individual topic
topic_freq = df['topics'].str.split(', ').explode().value_counts()

# Find the Top 5 most popular topics throughout months
top5_topics = topic_freq.head(3)

# Create a line plot to illustrate each topic's daily usage over time
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['date'] = df['createdAt'].dt.date

fig = go.Figure()

for topic in top5_topics.index:
    topic_df = df[df['topics'].str.contains(topic)]
    daily_counts = topic_df.groupby('date').size()
    fig.add_trace(go.Scatter(x=daily_counts.index, y=daily_counts, mode='lines', name=topic))

fig.update_layout(title='Top 3 Most Popular Topics Over Time',
                  xaxis_title='Date',
                  yaxis_title='Daily Usage',
                  legend_title='Topics')

fig.show()

## Usage of the Top 3 topics over time

In [352]:
# Get the frequency of each individual topic
topic_freq = df['topics'].str.split(', ').explode().value_counts().reset_index()
topic_freq.columns = ['topics', 'frequency']

print(topic_freq.head(5))
print(topic_freq.shape)


# Find the Top 5 most popular topics throughout months
# top5_topics = topic_freq.head(10)
# print(top5_topics)


# # Get the middle point of the column
# middle_point = topic_freq['frequency'].median()

# # Print the middle point
# print(middle_point)


# Create a line plot to illustrate each topic's daily usage over time
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['date'] = df['createdAt'].dt.date

fig = go.Figure()

for topic in top5_topics.index:
    topic_df = df[df['topics'].str.contains(topic)]
    daily_counts = topic_df.groupby('date').size()
    fig.add_trace(go.Scatter(x=daily_counts.index, y=daily_counts, mode='lines', name=topic))

fig.update_layout(title='Top 5 Most Popular Topics Over Time',
                  xaxis_title='Date',
                  yaxis_title='Daily Usage',
                  legend_title='Topics')

fig.show()



# # Get the frequency of each individual topic
# topic_freq = df['topics'].str.split(', ').explode().value_counts()
# print(topic_freq.info())
# print(topic_freq.shape)

# # Sort the topics in descending order and get the Top 5 most popular topics
# top5_topics = topic_freq.tail(371)
# print(top5_topics)

# # Create the Plotly bar plot
# fig = go.Figure()

# # Add the bar trace
# fig.add_trace(go.Bar(
#     x=top5_topics.index,  # Topics as x-axis
#     y=top5_topics.values,  # Frequency as y-axis
#     marker_color='rgb(67, 158, 213)',  # Custom bar color
#     text=top5_topics.values,  # Show the frequency on top of the bars
#     textposition='auto',  # Automatically position the text
# ))

# # Customize the layout
# fig.update_layout(
#     title='Top 5 Most Popular Topics',
#     xaxis_title='Topics',
#     yaxis_title='Frequency',
#     xaxis=dict(tickangle=-45),  # Rotate x-axis labels for better readability
#     yaxis=dict(title_standoff=20),  # Increase space between y-axis title and ticks
#     margin=dict(l=50, r=50, b=100, t=100),  # Add some margin for better spacing
#     plot_bgcolor='white',  # Set the plot background color
#     paper_bgcolor='white',  # Set the paper background color
# )

# # Show the plot
# fig.show()

                    topics  frequency
0  Artificial Intelligence       1868
1             Productivity       1743
2                Marketing        956
3                     Tech        886
4          Developer Tools        759
(371, 2)


### Clalculate the correlation between the Top 100 Topics and the VoteCounts

In [353]:
categories_df = df['topics'].str.get_dummies(', ')
df = pd.concat([df, categories_df], axis=1)
correlation_matrix = df.corr()
correlation_with_votes = correlation_matrix['votesCount'].drop(['votesCount', 'commentsCount'])

correlation_data = correlation_with_votes.reset_index()
correlation_data.columns = ['topics', 'Correlation with Votes']
# correlation_data.head(5)

#### Sort both the correlation and frequency tables by ascending alphabetical order

In [354]:

df1 = pd.DataFrame(topic_freq)
df1.sort_values(by='topics', inplace=True, ascending= True)
df1.reset_index(drop=True, inplace=True) 

df2 = pd.DataFrame(correlation_data)
df2.sort_values(by='topics', inplace=True, ascending= True)
df2.reset_index(drop=True, inplace=True) 

In [355]:

# Merge the tables using pd.concat
merged_df = pd.concat([df2, df1], axis=1)

# Rename the columns
merged_df.columns = ['topics_1', 'Correlation with Votes', 'topics', 'Frequency']
merged_df.head(5)

,topics_1,Correlation with Votes,topics,Frequency
0,3D Modeling,-0.007010,3D Modeling,17.0
1,3D Printer,-0.010312,3D Printer,6.0
2,A/B Testing,0.001022,A/B Testing,11.0
3,API,0.039022,API,172.0
4,ARKit,-0.007412,ARKit,3.0


In [356]:
# Drop the first 'topics' column
Corr_df = merged_df.drop(columns=['topics_1'])
Corr_df.sort_values(by='Frequency', inplace=True, ascending= False)
Corr_df.reset_index(drop=True, inplace=True) 
Top_100_Corr_df = Corr_df.head(100)

In [357]:
fig = px.bar(Top_100_Corr_df, x='topics', y='Correlation with Votes',
             title='Correlation Between topics and Votes',
             labels={'topics': 'Product Topics', 'Correlation with Votes': 'Correlation'},
             color='Correlation with Votes',
             color_continuous_scale='RdBu', 
             height=600)

fig.show()

### Cluster distribution of the all above average Topics

In [368]:
abv_avg_tpc_clsts = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Stats_Code/Data_Wrangling/Above_avg_Topics_Clusters.csv")


# Group the data by clusters and count the number of topics in each cluster
cluster_counts = abv_avg_tpc_clsts['Clusters'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Count']

# Create a modern and beautiful pie plot using Plotly
fig = px.pie(cluster_counts, values='Count', names='Cluster', title='Topic Clusters Distribution',
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Customize the layout for a better look
fig.update_traces(textinfo='percent+label', pull=[0.1, 0, 0, 0, 0], textfont_size=14,
                  marker=dict(line=dict(color='white', width=2)))

fig.update_layout(
    title_font=dict(size=24),
    legend=dict(title='Clusters', title_font_size=16, font_size=14),
    margin=dict(t=80, b=10, r=10, l=10),
    showlegend=True
)

# Show the plot
fig.show()

In [367]:
# average_frequency = df['votesCount'].mean()
# print(average_frequency)
# # Use boolean indexing to filter rows with Frequency above the mean
# rows_above_mean = df[df['votesCount'] > average_frequency]

# # Get the count of rows above the mean
# count_above_mean = len(rows_above_mean)
# print("Number of rows with Frequency above the mean:", count_above_mean)

# # Create a comma-separated list of topics from rows above the mean
# # topics_above_mean = ', '.join(rows_above_mean['topics'])

# # print("Topics with Frequency above the mean:", topics_above_mean)

75.40401218892839
Number of rows with Frequency above the mean: 1646


## Daily product creation thoughout Q1

In [359]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from matplotlib.dates import DateFormatter
import networkx as nx
from adjustText import adjust_text
import plotly.graph_objects as go
import plotly.express as px


In [360]:
import plotly.graph_objects as go

# Assuming your DataFrame is already defined as 'df'

# Group by date and count the number of products created each day
daily_product_creation = df.groupby('date').size().reset_index(name='count')

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_product_creation['date'], y=daily_product_creation['count'], mode='lines+markers'))
fig.update_layout(title='Daily Product Creation Throughout the Months', xaxis_title='Date', yaxis_title='Count')
fig.show()


## The most active days in each month

In [361]:
# Create a new column 'year-month-day' to store the date in the desired format
df['year-month-day'] = pd.to_datetime(df['createdAt']).dt.strftime('%Y-%m-%d')

# Group by 'Month' and 'year-month-day' and get the count of products for each day
most_active_days = df.groupby(['Month', 'year-month-day']).size().reset_index(name='count')

# Get the index of the maximum count for each month
idx = most_active_days.groupby('Month')['count'].idxmax()

# Use the index to extract the most active day for each month
dates_to_plot = most_active_days.loc[idx, 'year-month-day'].tolist()

# Filter the DataFrame to include only the most active days
most_active_days = most_active_days[most_active_days['year-month-day'].isin(dates_to_plot)]

# Create the bar plot using Plotly Express
fig = px.bar(most_active_days, x='year-month-day', y='count', title='Most Popular Days for Product Creation',
             labels={'year-month-day': 'Date', 'count': 'Number of Products'})

# Customize the appearance of the plot
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title_font=dict(size=14),
    yaxis_title_font=dict(size=14),
    title_font=dict(size=20),
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=60, r=20, t=80, b=60),
)

# Update the bar color and hover template
fig.update_traces(
    marker_color='rgb(68, 114, 196)',
    hovertemplate='<b>Date</b>: %{x}<br><b>Number of Products</b>: %{y}',
)

# Show the plot
fig.show()

## Hourly product creation for the most pupular days

In [362]:
# Create a Plotly figure
fig = go.Figure()

# Variables to track the minimum and maximum counts across all dates
min_count = float('inf')
max_count = float('-inf')

# Loop through each date and add a trace to the figure
for date in dates_to_plot:
    most_active_Day_df = df[df['createdAt'].dt.date == pd.to_datetime(date).date()]
    hour_counts = most_active_Day_df['hours'].value_counts().sort_index()

    # Update min_count and max_count
    min_count = min(min_count, hour_counts.min())
    max_count = max(max_count, hour_counts.max())

    fig.add_trace(go.Scatter(x=hour_counts.index, y=hour_counts.values, mode='lines+markers', line_shape='linear', name=date))

# Calculate a suitable step size for the Y-axis ticks
step_size = max(1, round((max_count - min_count) / 10))

# Update the layout with a white background and adjust Y-axis ticks
fig.update_layout(
    title='Number of Products Launched Each Hour',
    xaxis_title='Hour of the Day',
    yaxis_title='Number of Products',
    xaxis=dict(tickvals=list(range(24))),
    yaxis=dict(tickmode='array',  # Set the tick mode to 'array' for custom tick values
               tickvals=list(range(min_count, max_count + step_size, step_size)),  # Use custom tick values
               ),
    height=500,           # Adjust the plot height to provide more space for labels
    margin=dict(t=50),    # Add margin at the top for the title
    template='plotly_white'  # Use a white background template
)

# Update marker and line styles
fig.update_traces(marker=dict(size=8, line=dict(width=2, color='black')), line=dict(width=2))

# Show the plot
fig.show()

## The Top 5 most popular products for each month

In [363]:
# Assuming your DataFrame is already defined as 'df'

# Get the Top 5 most popular products based on VoteCount for each month
top5_products_per_month = df.groupby(df['createdAt'].dt.to_period('M')).apply(lambda x: x.nlargest(5, 'votesCount'))

top5_products_per_month[['name', 'votesCount', 'topics']].head(15)


c:\Users\alime\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



name  votesCount  \
createdAt                                       
2023-01   2703    Swapped Finance        3338   
          2704        HyperSwitch        2277   
          8116              Swimm        1455   
          1811        Piggy Magic        1352   
          18052       Originality        1225   
2023-02   49019             Bento        2410   
          55913          ChainGPT        2398   
          42407              Typo        2073   
          223917           Nas.io        1936   
          65810           WebWave        1858   
2023-03   340961  Chat by Copy.ai        5022   
          340962        Chat.D-ID        2318   
          325976            GPT-4        1851   
          317683         GoIT LMS        1828   
          324152          Collato        1499   

                                                             topics  
createdAt                                                            
2023-01   2703                                 Web3, Cryptocurrency  
          2704                    API, Open Source, User Experience  
          8116          Software Engineering, Developer Tools, Tech  
          1811    Design Tools, Productivity, Artificial Intelli...  
          18052             Marketing, SEO, Artificial Intelligence  
2023-02   49019   Social Network, Social Media, Website Builder,...  
          55913           Artificial Intelligence, Web3, Blockchain  
          42407      Productivity, Analytics, SaaS, Developer Tools  
          223917                                    SaaS, Community  
          65810   Design Tools, Website Builder, No-Code, Graphi...  
2023-03   340961   Productivity, Marketing, Artificial Intelligence  
          340962            Messaging, API, Artificial Intelligence  
          325976                            Artificial Intelligence  
          317683                   Education, Tech, Online Learning  
          324152        Productivity, SaaS, Artificial Intelligence

## The Top 5 most popular products for Q1

In [364]:
# Assuming your DataFrame is already defined as 'df'

# Get the Top 5 most popular products based on VoteCount for all months
top5_products_all_months = df.nlargest(5, 'votesCount')

top5_products_all_months[['name', 'votesCount', 'topics']].head()


,name,votesCount,topics
340961,Chat by Copy.ai,5022,"Productivity, Marketing, Artificial Intelligence"
2703,Swapped Finance,3338,"Web3, Cryptocurrency"
49019,Bento,2410,"Social Network, Social Media, Website Builder,..."
55913,ChainGPT,2398,"Artificial Intelligence, Web3, Blockchain"
340962,Chat.D-ID,2318,"Messaging, API, Artificial Intelligence"


## The Top 3 most popular topic

In [365]:
# Get the frequency of each individual topic
topic_freq = top5_products_all_months['topics'].str.split(', ').explode().value_counts()

# Find the Top 5 most popular topics throughout months
top5_topics = topic_freq.head(3)
top5_topics.head()

Artificial Intelligence    3
Web3                       2
Productivity               1
Name: topics, dtype: int64